<a href="https://colab.research.google.com/github/sathishbandaru/minorproject/blob/main/Copy_of_mini_project_text_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [3]:
cxr_df=pd.read_csv('/content/drive/My Drive/Mini Project/cxrfolder/cxr_df.csv')
labeled_df=pd.read_csv('/content/drive/My Drive/Mini Project/cxrfolder/labeled_reports.csv')

In [4]:
labeled_df['path']=cxr_df['path']

In [5]:
sample_df=labeled_df.sample(10)

In [6]:
labeled_df.columns

Index(['Report Impression', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices', 'No Finding', 'path'],
      dtype='object')

In [7]:
atelectasis_df=labeled_df[labeled_df['Atelectasis']==1]

In [8]:
len(atelectasis_df)

31042

In [9]:
reports=atelectasis_df['Report Impression']
len(reports)

31042

In [10]:
import spacy
from tqdm import tqdm

def extract_atelectasis_observations(reports):
    nlp = spacy.load("en_core_web_sm")

    atelectasis_synonyms = [
        "partial lung collapse",
        "lung collapse",
        "partial lung collapses",
        "bibasilar atelectatic changes persist",
        "mild left basal atelectasis",
        "lung volume loss",
        "collapsed lung segment",
        "alveolar collapse",
        "pulmonary collapse",
        "subsegmental atelectasis",
        "compression atelectasis",
        "obstructive atelectasis",
        "resorption atelectasis",
        "atelectasis",
        "atelectatic"
    ]

    atelectasis_observations = []

    for report in tqdm(reports):
        try:
            doc = nlp(report)
        except Exception as e:
            print(f"Error processing report: {report}")
            print(f"Error details: {e}")
            continue

        atelectasis_info = ""
        for sent in doc.sents:
            try:

                if any(synonym in sent.text.lower() for synonym in atelectasis_synonyms):

                    independent_clause = None
                    for clause in sent.noun_chunks:
                        if "atelectasis" in clause.text.lower() or any(synonym in clause.text.lower() for synonym in atelectasis_synonyms):
                            independent_clause = clause.text
                            break


                    atelectasis_info += (independent_clause or sent.text) + " "
            except Exception as e:
                print(f"Error processing sentence: {sent.text}")
                print(f"Error details: {e}")

        atelectasis_observations.append(atelectasis_info.strip() or "No atelectasis information")

    return atelectasis_observations


result = extract_atelectasis_observations(reports)


100%|██████████| 31042/31042 [07:53<00:00, 65.56it/s]


In [11]:
len(result)

31042

In [12]:
atelectasis_df['text']=result

<ipython-input-12-0e0f2213d1ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atelectasis_df['text']=result


In [13]:
atelectasis_df['text'].value_counts()

,count
text,
atelectasis,6251
No atelectasis information,1997
bibasilar atelectasis,1072
lower lobe atelectasis,831
compressive atelectasis,806
...,...
substantial basal atelectasis,1
upper lobe rounded atelectasis,1
"Mild retrocardiac opacity unchanged, likely atelectasis.",1


In [14]:
atelectasis_df1=atelectasis_df[atelectasis_df['text']!="atelectasis"]
atelectasis_df1=atelectasis_df1[atelectasis_df1['text']!="No atelectasis information"]

In [15]:
atelectasis_df1['text'].value_counts()

,count
text,
bibasilar atelectasis,1072
lower lobe atelectasis,831
compressive atelectasis,806
Bibasilar atelectasis,805
basilar atelectasis,585
...,...
substantial basal atelectasis,1
upper lobe rounded atelectasis,1
"Mild retrocardiac opacity unchanged, likely atelectasis.",1


In [16]:
atelectasis_df1=atelectasis_df1.sample(frac=1)

In [17]:
atelectasis_df_final=atelectasis_df1[['path','text','Report Impression']]

In [18]:
atelectasis_df_code=atelectasis_df1[['path','text']]

In [19]:
atelectasis_df_code.head(9)

,path,text
83861,../input/curated-cxr-report-generation-dataset...,mild right middle lobe atelectasis
35194,../input/curated-cxr-report-generation-dataset...,basal atelectasis
15858,../input/curated-cxr-report-generation-dataset...,bibasilar atelectasis
5722,../input/curated-cxr-report-generation-dataset...,retrocardiac atelectasis
7182,../input/curated-cxr-report-generation-dataset...,the atelectasis
22737,../input/curated-cxr-report-generation-dataset...,mild retrocardiac atelectasis
17301,../input/curated-cxr-report-generation-dataset...,lower lobe atelectasis
471,../input/curated-cxr-report-generation-dataset...,underlying atelectasis
34223,../input/curated-cxr-report-generation-dataset...,left basal atelectasis


In [20]:
atelectasis_df_final.to_csv('atelectasis.csv',index=False)
atelectasis_df_code.to_csv('atelectasis_captions.txt',index=False)

In [21]:
atelectasis_df_full=atelectasis_df_final[['path','Report Impression']]

In [22]:
atelectasis_df_full.to_csv('atelectasis.txt',index=False)

In [23]:
len(atelectasis_df_full)

22794

In [24]:
atelectasis_df_final.head()

,path,text,Report Impression
83861,../input/curated-cxr-report-generation-dataset...,mild right middle lobe atelectasis,"Other than mild right middle lobe atelectasis,..."
35194,../input/curated-cxr-report-generation-dataset...,basal atelectasis,"As compared to the previous radiograph, there ..."
15858,../input/curated-cxr-report-generation-dataset...,bibasilar atelectasis,NG tube has been advanced in distal stomach. E...
5722,../input/curated-cxr-report-generation-dataset...,retrocardiac atelectasis,Comparison to ___. No relevant change is noted...
7182,../input/curated-cxr-report-generation-dataset...,the atelectasis,"As compared to the previous radiograph, there ..."
